In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import ta

In [2]:
BTC = pd.read_hdf(f'Data/BTCUSDT.h5', key='my_data')
ETH = pd.read_hdf(f'Data/ETHUSDT.h5', key='my_data')

In [3]:
BTC['Date'] = pd.to_datetime(BTC['Date'])
ETH['Date'] = pd.to_datetime(ETH['Date'])
BTC = BTC.rename(columns={'Open time': 'Date', 'Close': 'BTC'})
ETH = ETH.rename(columns={'Open time': 'Date', 'Close': 'ETH'})

In [4]:
merged_data=pd.merge(BTC,ETH,on='Date',how='left').dropna()
merged_data=merged_data.set_index('Date')

In [5]:
merged_data['BTC']=merged_data['BTC'].astype(float)
merged_data['ETH']=merged_data['ETH'].astype(float)

In [6]:
def Technical_indicators(Merged,target):
    for i in Merged:
        Merged['Log_return_'+i]=np.log(Merged[i])/np.log(Merged[i].shift(5))-1

        
    window_4_hr=60*4
    window_30_M=30
    window_1_day=60*24
    window_2_day=60*24*2
    window_2_weeks=60*24*14

    Merged['RSI 30M'] = ta.momentum.RSIIndicator(Merged[target], window=window_30_M).rsi()
    Merged['RSI 30M_EMA'] = Merged['RSI 30M'].ewm(span=20, adjust=False).mean()
    Merged['RSI 4hr'] = ta.momentum.RSIIndicator(Merged[target], window=window_4_hr).rsi()
    Merged['RSI 1D'] = ta.momentum.RSIIndicator(Merged[target], window=window_1_day).rsi()
    Merged['RSI 2D'] = ta.momentum.RSIIndicator(Merged[target], window=window_2_day).rsi()
    Merged['RSI_diff'] = (Merged['RSI 2D']-Merged['RSI 1D'])
    Merged['RSI 1D_diff'] =Merged['RSI 1D'].diff()
    Merged['RSI 2Weeks'] = ta.momentum.RSIIndicator(Merged[target], window=window_2_weeks).rsi()
    Merged['ema 4HR'] = Merged[target].ewm(span=window_4_hr, adjust=False).mean()
    Merged['ema 1D'] = Merged[target].ewm(span=window_1_day, adjust=False).mean()
    Merged['ema 2W'] = Merged[target].ewm(span=window_2_weeks, adjust=False).mean()
    Merged['SMA_50'] = ta.trend.sma_indicator(Merged[target], window=50)
    Merged['SMA_200'] = ta.trend.sma_indicator(Merged[target], window=200)
    Merged['SMA_12HR'] = ta.trend.sma_indicator(Merged[target], window=60*12)
    Merged['RSI'] = ta.momentum.RSIIndicator(Merged[target], window=30).rsi()
    Merged['MACD'] = ta.trend.MACD(Merged[target], window_fast=window_30_M, window_slow=60).macd()
    Merged['MACD30M'] = Merged['MACD'].ewm(span=24*60, adjust=False).mean()
    Merged['bb_upper'] = ta.volatility.BollingerBands(Merged[target], window=60).bollinger_hband()
    Merged['bb_lower'] = ta.volatility.BollingerBands(Merged[target], window=60).bollinger_lband()
    Merged['SMA100'] = Merged[target].rolling(window=100).mean()
    rolling_stats = Merged[target].rolling(window=20).agg(['mean', 'std'])
    Merged['Middle_Band'] = rolling_stats['mean']
    Merged['Upper_Band'] = rolling_stats['mean'] + 2 * rolling_stats['std']
    Merged['Lower_Band'] = rolling_stats['mean'] - 2 * rolling_stats['std']
    # Merged['+DM'] = np.where((Merged[target].diff() > 0), Merged[target].diff(), 0)
    # Merged['-DM'] = np.where((Merged[target].diff() < 0), abs(Merged[target].diff()), 0)
    # Merged['TR'] = np.maximum((Merged[target] - Merged[target].shift()), Merged[target].diff())
    # Merged['ATR'] = Merged['TR'].rolling(window=60).mean()
    # Merged['+DI'] = 100 * (Merged['+DM']/Merged['ATR']).rolling(window=60).mean()
    # Merged['-DI'] = 100 * (Merged['-DM']/Merged['ATR']).rolling(window=60).mean()
    # Merged['DX'] = (abs(Merged['+DI'] - Merged['-DI']) / abs(Merged['+DI'] + Merged['-DI'])) * 100
    # Merged['ADX'] = Merged['DX'].rolling(window=60).mean()

    return Merged

In [7]:
target='BTC'
merged_data=Technical_indicators(merged_data,target)
merged_data['Objective']=np.where(merged_data[target].shift(-30)-merged_data[target]>100,1,0)
merged_data=merged_data.dropna()

In [8]:
data=merged_data.copy()

In [9]:
data

,BTC,ETH,Log_return_BTC,Log_return_ETH,RSI 30M,RSI 30M_EMA,RSI 4hr,RSI 1D,RSI 2D,RSI_diff,...,RSI,MACD,MACD30M,bb_upper,bb_lower,SMA100,Middle_Band,Upper_Band,Lower_Band,Objective
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-14 23:59:00,39144.50,1231.62,-0.000191,0.001409,54.228581,57.978643,50.255693,51.217069,50.711155,-0.505914,...,54.228581,83.332276,13.118668,39300.305846,38772.814154,38902.6463,39146.5555,39389.342317,38903.768683,1
2021-01-15 00:00:00,39207.17,1236.95,-0.000103,0.001903,56.378185,57.826219,50.540057,51.259075,50.729835,-0.529240,...,56.378185,83.423514,13.216246,39307.110917,38774.865083,38908.2493,39156.8210,39391.059094,38922.582906,1
2021-01-15 00:01:00,39031.95,1226.15,-0.000537,0.000697,49.635898,57.046188,49.741714,51.135880,50.676120,-0.459760,...,49.635898,77.770005,13.305842,39307.300301,38775.182032,38911.5689,39158.2495,39388.868058,38927.630942,1
2021-01-15 00:02:00,39184.01,1233.49,-0.000320,0.001464,54.518847,56.805489,50.424161,51.237655,50.721417,-0.516238,...,54.518847,77.314884,13.394681,39311.904085,38774.827915,38916.5878,39168.9440,39381.997892,38955.890108,1
2021-01-15 00:03:00,39360.01,1239.87,0.000116,0.001467,59.249441,57.038246,51.194428,51.355007,50.773761,-0.581246,...,59.249441,82.323889,13.490350,39328.790880,38767.820453,38922.9304,39187.5580,39398.994141,38976.121859,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-12 12:17:00,61108.55,2928.97,-0.000022,0.000098,46.021718,46.975259,50.862130,49.847016,49.648662,-0.198353,...,46.021718,-10.502323,2.815561,61202.907086,61096.933247,61159.7437,61130.2115,61183.990556,61076.432444,0
2024-05-12 12:18:00,61117.63,2928.83,0.000001,0.000101,47.143553,46.991287,50.989426,49.864208,49.655776,-0.208432,...,47.143553,-10.621877,2.796911,61201.912947,61095.848053,61157.6672,61128.7935,61182.315766,61075.271234,0
2024-05-12 12:19:00,61116.61,2928.30,0.000021,0.000086,47.029968,46.994971,50.974530,49.862287,49.654988,-0.207299,...,47.029968,-10.739948,2.778123,61200.651665,61094.879335,61156.1641,61127.3240,61180.469662,61074.178338,0


In [10]:
feature_names = data.columns[~data.columns.isin(['BTC', 'Objective'])].tolist()
target_name = 'BTC'
print(feature_names)
print(target)

['ETH', 'Log_return_BTC', 'Log_return_ETH', 'RSI 30M', 'RSI 30M_EMA', 'RSI 4hr', 'RSI 1D', 'RSI 2D', 'RSI_diff', 'RSI 1D_diff', 'RSI 2Weeks', 'ema 4HR', 'ema 1D', 'ema 2W', 'SMA_50', 'SMA_200', 'SMA_12HR', 'RSI', 'MACD', 'MACD30M', 'bb_upper', 'bb_lower', 'SMA100', 'Middle_Band', 'Upper_Band', 'Lower_Band']
BTC


In [11]:
X_train = data[feature_names].iloc[:100000]
y_train = data[target_name].iloc[:100000]
X_test = data[feature_names].iloc[100000:]
y_test = data[target_name].iloc[100000:]

In [12]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [19]:
use_model=data.iloc[100000:]

In [29]:
use_model['signal'] = np.where(model.predict(use_model[feature_names]) > use_model[target_name], 1, -1)


C:\Users\adnan\AppData\Local\Temp\ipykernel_10508\2651217900.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
import vectorbt as vbt
buy_signals = use_model['signal']==1
sell_signals = use_model['signal']==-1

# Create the portfolio
pf = vbt.Portfolio.from_signals(use_model['BTC'],
                                    #    entries=buy_signals,
                                    #    exits=sell_signals,
                                       tp_stop=0.1,
                                       sl_stop=0.1,
                                       sl_trail=True,
                                       short_entries=sell_signals,
                                       short_exits=buy_signals,
                                       fees=0.005,init_cash=10000)
# Analyze the results
portfolio_perf = pf.stats()
print(portfolio_perf)

Start                         2021-03-25 13:28:00
End                           2024-05-12 12:21:00
Period                                    1646391
Start Value                               10000.0
End Value                                0.000154
Total Return [%]                       -99.999998
Benchmark Return [%]                     19.89798
Max Gross Exposure [%]                      100.0
Total Fees Paid                       9829.569989
Max Drawdown [%]                        99.999998
Max Drawdown Duration                   1646390.0
Total Trades                                 1757
Total Closed Trades                          1757
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                             0.739898
Best Trade [%]                           1.820165
Worst Trade [%]                         -7.573784
Avg Winning Trade [%]                    0.507011
Avg Losing Trade [%]                    -1.034661


c:\Users\adnan\AppData\Local\Programs\Python\Python39\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\adnan\AppData\Local\Programs\Python\Python39\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\adnan\AppData\Local\Programs\Python\Python39\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\adnan\AppData\Local\Programs\Python\Python39\lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



In [28]:
#pf.plot().show()